In [2]:
import geopandas as gpd
import pandas as pd
import xarray as xr

In [70]:
province = gpd.read_file(
    r"C:\Users\MUICT\Desktop\STONES\NASA_data\tha_admbnda_adm1_rtsd_20190221.shp")
province = province.to_crs("EPSG:4326")
# 1. Load the Thailand shapefile

In [4]:
 # national boundary


for var in ['vap', ]:
    temp_df = pd.DataFrame()
    for year in range(2002, 2024):
        try:
            ds = xr.open_dataset(f"pre_data/TerraClimate_{var}_{year}.nc")
        except:
            print(f"{var} No data available for year {year}")
            continue

        raster = ds[var]
        raster.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        raster.rio.write_crs("EPSG:4326", inplace=True)
        raster_thailand = raster.rio.clip(thailand.geometry, thailand.crs)
        df = raster_thailand.to_dataframe(name=var).reset_index()
        df = df.loc[df[var].notnull()].reset_index(drop=True)
        df_gdp = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df["lon"], df["lat"]),
            crs="EPSG:4326"
        )

        # spatial join: assign each pixel to a province
        df_joined = gpd.sjoin(
            df_gdp,
            province[["ADM1_EN", "geometry"]],
            # how="inner",
            # predicate="within"
        )

        # province mean
        province_mean = (
            df_joined
            .groupby(["ADM1_EN", 'time'])[var]
            .mean()
            .reset_index()
            .rename(columns={"ADM1_EN": "province", var: f"{var}"})
        )
        province_mean['month'] = province_mean['time'].dt.month
        province_mean['year'] = province_mean['time'].dt.year
        province_mean = province_mean.drop(columns=['time'])
        temp_df = pd.concat([temp_df, province_mean], axis=0)
    temp_df.to_csv(f'data/terraclimate_{var}_province_mean.csv', index=False)



vap No data available for year 2002
vap No data available for year 2011


In [73]:

for var in ['ppt', ]:
    temp_df = pd.DataFrame()
    for year in [2024]:
        try:
            ds = xr.open_dataset(f"pre_data/TerraClimate_{var}_{year}.nc")
        except:
            print(f"{var} No data available for year {year}")
            continue

        raster = ds[var]
        raster.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        raster.rio.write_crs("EPSG:4326", inplace=True)
        raster_thailand = raster.rio.clip(thailand.geometry, thailand.crs)
        df = raster_thailand.to_dataframe(name=var).reset_index()
        df = df.loc[df[var].notnull()].reset_index(drop=True)
        df_gdp = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df["lon"], df["lat"]),
            crs="EPSG:4326"
        )

        # spatial join: assign each pixel to a province
        df_joined = gpd.sjoin(
            df_gdp,
            province[["ADM1_EN", "geometry"]],
            # how="inner",
            # predicate="within"
        )

        # province mean
        province_mean = (
            df_joined
            .groupby(["ADM1_EN", 'time'])[var]
            .mean()
            .reset_index()
            .rename(columns={"ADM1_EN": "province", var: f"{var}"})
        )
        province_mean['month'] = province_mean['time'].dt.month
        province_mean['year'] = province_mean['time'].dt.year
        province_mean = province_mean.drop(columns=['time'])
        temp_df = pd.concat([temp_df, province_mean], axis=0)
    temp_df.to_csv(f'data/terraclimate_{var}_province_mean_2024.csv', index=False)



In [75]:
for var in ['tmax', 'tmin', 'ppt', ]:
    df = pd.read_csv(f'data/terraclimate_{var}_province_mean_2024.csv')
    df = df[['year', 'month', 'province', var]]
    df.to_csv(f'data/terraclimate_{var}_province_mean_2024.csv', index=False)

In [76]:
final_df = pd.read_csv(f'data/terraclimate_vap_province_mean.csv')
for var in ['tmax', 'tmin', 'ppt', 'soil', 'ws']:
    df_var = pd.read_csv(f'data/terraclimate_{var}_province_mean.csv')
    final_df = pd.merge(final_df, df_var, on=['province', 'month', 'year'], how='left')
final_df = final_df.rename(columns={
    'tmax': 'TMAX',
    'tmin': 'TMIN',
    'ppt': 'PPT',
    'soil': 'SOIL',
    'vap': 'VAP',
    'ws': 'WS',
})


In [80]:

# for var in ['tmax','tmin','ppt','soil','vap','vpd','ws']:
#     df = pd.read_csv(f'data/Final_data_province_mean_{var}.csv')
#     df = df[df['year']>=2003]
#     df.to_csv(f'data/Final_data_province_mean_{var}_2003onward.csv', index=False)
merged_df = pd.read_csv('data/Final_data_province_mean_ppt_2003onward.csv')
merged_df = merged_df.replace('Loburi', 'Lop Buri')
for var in ['tmax', 'tmin', 'soil', 'vap', 'vpd', 'ws']:
    df_var = pd.read_csv(f'data/Final_data_province_mean_{var}_2003onward.csv')
    df_var = df_var.replace('Loburi', 'Lop Buri')
    merged_df = pd.merge(merged_df, df_var, on=['province', 'month', 'year'], how='left')
merged_df['TAVG'] = (merged_df['tmax'] + merged_df['tmin']) / 2
merged_df = merged_df.rename(columns={
    'tmax': 'TMAX',
    'tmin': 'TMIN',
    'ppt': 'PPT',
    'soil': 'SOIL',
    'vap': 'VAP',
    'ws': 'WS',
    'vpd': 'VPD',
})
merged_df

,year,month,province,PPT,TMAX,TMIN,SOIL,VAP,VPD,WS,TAVG
0,2003,1,Amnat Charoen,0.117361,30.417500,16.233681,84.517708,1.747604,1.361509,1.975382,23.325590
1,2003,2,Amnat Charoen,30.798264,33.323403,19.535972,64.366319,2.110201,1.607610,1.118299,26.429688
2,2003,3,Amnat Charoen,50.106250,34.351285,22.163542,50.811806,2.210365,1.862704,1.683854,28.257413
3,2003,4,Amnat Charoen,63.870139,36.742951,24.875069,42.203819,2.726635,1.969497,1.506250,30.809010
4,2003,5,Amnat Charoen,219.253125,35.316979,25.015278,90.012500,3.026176,1.447736,1.561667,30.166128
...,...,...,...,...,...,...,...,...,...,...,...
20323,2024,8,Yasothon,149.879592,32.494388,25.045408,NaN,NaN,NaN,NaN,28.769898
20324,2024,9,Yasothon,268.469898,31.745918,24.327041,NaN,NaN,NaN,NaN,28.036480
20325,2024,10,Yasothon,74.651531,32.167347,22.931633,NaN,NaN,NaN,NaN,27.549490
20326,2024,11,Yasothon,9.287755,31.913265,20.434694,NaN,NaN,NaN,NaN,26.173980


In [190]:
merged_df.province.unique()

array(['Amnat Charoen', 'Ang Thong', 'Bangkok', 'Bueng Kan', 'Buri Ram',
       'Chachoengsao', 'Chai Nat', 'Chaiyaphum', 'Chanthaburi',
       'Chiang Mai', 'Chiang Rai', 'Chon Buri', 'Chumphon', 'Kalasin',
       'Kamphaeng Phet', 'Kanchanaburi', 'Khon Kaen', 'Krabi', 'Lampang',
       'Lamphun', 'Loei', 'Lop Buri', 'Mae Hong Son', 'Maha Sarakham',
       'Mukdahan', 'Nakhon Nayok', 'Nakhon Pathom', 'Nakhon Phanom',
       'Nakhon Ratchasima', 'Nakhon Sawan', 'Nakhon Si Thammarat', 'Nan',
       'Narathiwat', 'Nong Bua Lam Phu', 'Nong Khai', 'Nonthaburi',
       'Pathum Thani', 'Pattani', 'Phangnga', 'Phatthalung', 'Phayao',
       'Phetchabun', 'Phetchaburi', 'Phichit', 'Phitsanulok',
       'Phra Nakhon Si Ayutthaya', 'Phrae', 'Phuket', 'Prachin Buri',
       'Prachuap Khiri Khan', 'Ranong', 'Ratchaburi', 'Rayong', 'Roi Et',
       'Sa Kaeo', 'Sakon Nakhon', 'Samut Prakan', 'Samut Sakhon',
       'Samut Songkhram', 'Saraburi', 'Satun', 'Si Sa Ket', 'Sing Buri',
       'Songkhla', '

In [192]:

merged_df = merged_df.sort_values(by=['province', 'year', 'month']).reset_index(drop=True)
merged_df = merged_df.replace('Bueng Kan','Nong Khai' )
merged_df = merged_df.groupby(['province', 'year', 'month']).mean().reset_index()
merged_df.province.unique()

array(['Amnat Charoen', 'Ang Thong', 'Bangkok', 'Buri Ram',
       'Chachoengsao', 'Chai Nat', 'Chaiyaphum', 'Chanthaburi',
       'Chiang Mai', 'Chiang Rai', 'Chon Buri', 'Chumphon', 'Kalasin',
       'Kamphaeng Phet', 'Kanchanaburi', 'Khon Kaen', 'Krabi', 'Lampang',
       'Lamphun', 'Loei', 'Lop Buri', 'Mae Hong Son', 'Maha Sarakham',
       'Mukdahan', 'Nakhon Nayok', 'Nakhon Pathom', 'Nakhon Phanom',
       'Nakhon Ratchasima', 'Nakhon Sawan', 'Nakhon Si Thammarat', 'Nan',
       'Narathiwat', 'Nong Bua Lam Phu', 'Nong Khai', 'Nonthaburi',
       'Pathum Thani', 'Pattani', 'Phangnga', 'Phatthalung', 'Phayao',
       'Phetchabun', 'Phetchaburi', 'Phichit', 'Phitsanulok',
       'Phra Nakhon Si Ayutthaya', 'Phrae', 'Phuket', 'Prachin Buri',
       'Prachuap Khiri Khan', 'Ranong', 'Ratchaburi', 'Rayong', 'Roi Et',
       'Sa Kaeo', 'Sakon Nakhon', 'Samut Prakan', 'Samut Sakhon',
       'Samut Songkhram', 'Saraburi', 'Satun', 'Si Sa Ket', 'Sing Buri',
       'Songkhla', 'Sukhothai', '

In [193]:
bio_clim_df = pd.DataFrame()
for province in merged_df['province'].unique():
    province_df = merged_df[merged_df['province'] == province]
    province_df['rolling_sum_PPT'] = province_df['PPT'].rolling(window=3).sum().shift(-2)
    province_df['rolling_avg_TAVG'] = province_df['TAVG'].rolling(window=3).mean().shift(-2)
    bio_clim = {}
    for i in range(province_df['year'].nunique()):
        if i + 2003 > 2023:
            break
        temp = province_df.iloc[12 * i:12 * (i + 1)].copy().reset_index(drop=True)
        # print(temp.head(15))
        bio_clim['year'] = i+2003
        bio_clim['province'] = province
        bio_clim['Bio 1'] = temp['TAVG'].mean()
        bio_clim['Bio 2'] = (temp['TMAX'] - temp['TMIN']).sum() / 12
        bio_clim['Bio 4a'] = ((temp['TAVG'] + 273.15).std() / (bio_clim['Bio 1'] + 273.15)) * 100
        bio_clim['Bio 4'] = temp['TAVG'].std() * 100
        bio_clim['Bio 5'] = temp['TMAX'].max()
        bio_clim['Bio 6'] = temp['TMIN'].min()
        bio_clim['Bio 7'] = bio_clim['Bio 5'] - bio_clim['Bio 6']
        wettest_quarter_month = temp['rolling_sum_PPT'].idxmax()
        driest_quarter_month = temp['rolling_sum_PPT'].idxmin()
        warmest_quarter_month = temp['rolling_avg_TAVG'].idxmax()
        coldest_quarter_month = temp['rolling_avg_TAVG'].idxmin()
        # print(wettest_quarter_month,driest_quarter_month)
        bio_clim['Bio 8'] = temp['TAVG'].iloc[wettest_quarter_month:wettest_quarter_month+3].mean()
        bio_clim['Bio 9'] = temp['TAVG'].iloc[driest_quarter_month:driest_quarter_month+3].mean()
        bio_clim['Bio 10'] = temp['TAVG'].iloc[warmest_quarter_month:warmest_quarter_month+3].mean()
        bio_clim['Bio 11'] = temp['TAVG'].iloc[coldest_quarter_month:coldest_quarter_month+3].mean()
        bio_clim['Bio 12'] = temp['PPT'].sum()
        bio_clim['Bio 13'] = temp['PPT'].max()
        bio_clim['Bio 14'] = temp['PPT'].min()
        bio_clim['Bio 15'] = (temp['PPT'].std() / (1 + bio_clim['Bio 12'] / 12)) * 100
        bio_clim['Bio 16'] = temp['rolling_sum_PPT'].iloc[wettest_quarter_month]
        bio_clim['Bio 17'] = temp['rolling_sum_PPT'].iloc[driest_quarter_month]
        bio_clim['Bio 18'] = temp['rolling_sum_PPT'].iloc[warmest_quarter_month]
        bio_clim['Bio 19'] = temp['rolling_sum_PPT'].iloc[coldest_quarter_month]

        bio_clim_df = pd.concat([bio_clim_df, pd.DataFrame([bio_clim])], axis=0)

bio_clim_df.to_csv('data/Final_data_province_mean_bioclim.csv', index=False)

C:\Users\MUICT\AppData\Local\Temp\ipykernel_34936\2983613467.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  province_df['rolling_sum_PPT'] = province_df['PPT'].rolling(window=3).sum().shift(-2)
C:\Users\MUICT\AppData\Local\Temp\ipykernel_34936\2983613467.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  province_df['rolling_avg_TAVG'] = province_df['TAVG'].rolling(window=3).mean().shift(-2)
C:\Users\MUICT\AppData\Local\Temp\ipykernel_34936\2983613467.py:4: SettingWithCopyWarning: 
A value is trying t

In [196]:
final_final_df = pd.read_csv('data/Final_data_province_mean_ppt_2003onward.csv')
final_final_df = final_final_df.replace({'Loburi': 'Lop Buri','Bueng Kan':'Nong Khai' })
final_final_df = final_final_df.groupby(['province', 'year', 'month']).mean().reset_index()

for var in ['TMAX', 'TMIN', 'SOIL', 'VAP', 'VPD', 'WS']:
    df_var = pd.read_csv(f'data/Final_data_province_mean_{var}_2003onward.csv')
    df_var = df_var.replace({'Loburi': 'Lop Buri','Bueng Kan':'Nong Khai' })
    df_var = df_var.groupby(['province', 'year', 'month']).mean().reset_index()
    final_final_df = pd.merge(final_final_df, df_var, on=['province','month', 'year'], how='left')
final_final_df =final_final_df[final_final_df['year']!=2024]
final_final_df = pd.merge(final_final_df,
                          bio_clim_df,
                          on=['province', 'year'],
                          how='left')
final_final_df = final_final_df.rename(columns={
    'tmax': 'TMAX',
    'tmin': 'TMIN',
    'ppt': 'PPT',
    'soil': 'SOIL',
    'vap': 'VAP',
    'ws': 'WS',
    'vpd': 'VPD',
})
final_final_df.to_csv('climate_data_terra_2003_2023_bio_76prov.csv', index=False)